<a href="https://colab.research.google.com/github/saeidfarahan/Machine-Learning/blob/main/gRPC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [23]:
# نصب کتابخانه‌های gRPC برای ارتباط سرور و کلاینت
!pip install grpcio grpcio-tools


In [24]:
# تعریف محتوای فایل proto (توضیح روش ارتباط سرور و کلاینت)
proto_content = """
syntax = "proto3";

package greet;

// تعریف سرویسی با نام Greeter
service Greeter {
  // تابعی که کلاینت فراخوانی می‌کند
  rpc SayHello (HelloRequest) returns (HelloReply) {}
}

// ساختار داده برای درخواست کلاینت (پیام ارسالی از کلاینت)
message HelloRequest {
  string name = 1;  // نام ارسالی از کلاینت
}

// ساختار داده برای پاسخ سرور (پیام ارسالی از سرور)
message HelloReply {
  string message = 1;  // متن خوشامدگویی بازگشتی از سرور
}
"""

# ایجاد و نوشتن محتوای بالا در فایل greet.proto
with open("greet.proto", "w") as f:
    f.write(proto_content)


In [25]:
# تولید کدهای پایتونی موردنیاز کلاینت و سرور از فایل greet.proto
!python -m grpc_tools.protoc -I. --python_out=. --grpc_python_out=. greet.proto


In [26]:
# کتابخانه‌های موردنیاز
import grpc
from concurrent import futures
import greet_pb2
import greet_pb2_grpc

# تعریف کلاس سرور با ارث‌بری از کلاس تولیدشده توسط proto
class GreeterServicer(greet_pb2_grpc.GreeterServicer):
    # پیاده‌سازی تابع SayHello برای پاسخ به کلاینت‌ها
    def SayHello(self, request, context):
        # ارسال پاسخ به کلاینت
        return greet_pb2.HelloReply(message=f'Hello, {request.name}! Welcome to gRPC.')

# تابع اصلی اجرای سرور
def serve():
    # ساخت سرور با حداکثر ۱۰ thread برای پاسخ به درخواست‌ها
    server = grpc.server(futures.ThreadPoolExecutor(max_workers=10))

    # اضافه کردن سرویس GreeterServicer به سرور
    greet_pb2_grpc.add_GreeterServicer_to_server(GreeterServicer(), server)

    # مشخص کردن پورت 50051 برای شنود درخواست‌های ورودی
    server.add_insecure_port('[::]:50051')

    # اجرای سرور
    server.start()
    print("Server started on port 50051.")

    # منتظر ماندن سرور تا دریافت درخواست‌ها
    server.wait_for_termination()

# اجرای سرور در پس‌زمینه با استفاده از threading
import threading
server_thread = threading.Thread(target=serve, daemon=True)
server_thread.start()


Server started on port 50051.


In [28]:
# واردکردن کتابخانه‌های gRPC
import grpc
import greet_pb2
import greet_pb2_grpc

# تعریف تابع کلاینت برای ارسال درخواست به سرور
def run_client(name):
    # ایجاد اتصال به سرور روی آدرس localhost و پورت 50051
    channel = grpc.insecure_channel('localhost:50051')

    # ایجاد Stub برای فراخوانی توابع سرور
    stub = greet_pb2_grpc.GreeterStub(channel)

    # فراخوانی تابع SayHello با ارسال نام به سرور و گرفتن پاسخ
    response = stub.SayHello(greet_pb2.HelloRequest(name=name))

    # نمایش پاسخ سرور
    print("Server response:", response.message)

# فراخوانی کلاینت با ارسال نام "Saeid"
run_client("SAEID")


Server response: Hello, SAEID! Welcome to gRPC.
